### 用不到50行的Python代码构建最小的区块链

see https://app.yinxiang.com/shard/s21/nl/5701630/44d472fc-abe0-42c0-8c0a-78cd0977218b

在区块链中，每个块都存储一个时间戳和一个索引。在SnakeCoin中，需要把两者都存储起来。为了确保整个区块链的完整性，每个块都有一个自动识别散列。与比特币一样，每个块的散列将是**块索引、时间戳、数据和前块哈希的加密哈希**。数据可以是你想要的任何东西。

In [1]:
import hashlib as hasher

class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hash_block()
        #print(self.hash)

    def hash_block(self):
        sha = hasher.sha256()
        block_str = str(self.index) + str(self.timestamp) + str(self.data) + str(self.previous_hash)
        sha.update(block_str.encode('utf-8'))
        return sha.hexdigest()

块链的第一个区块是如何到达那里的呢？不得不说，第一个块，或者说是**起源块**，它是一个特殊的块。在很多情况下，它是手动添加的，或者有独特的逻辑允许添加。

下面将创建一个函数简单地返回一个起源块以便产生第一个区块。这个块是索引0，它具有任意的数据值和“前一个哈希”参数中的任意值。

In [2]:
import datetime as date

def create_genesis_block():
    # Manually construct a block with
    # index zero and arbitrary previous hash
    return Block(0, date.datetime.now(), "Genesis Block", "0")

接下来需要一个函数，以便在区块链中生成后续的块。
- 这个函数将把链中的前一个块作为参数，创建要生成的块的数据，并使用适当的数据返回新块。
- 当新的块哈希信息来自前面的块时，区块链的完整性会随着每个新块而增加。

如果不这样做，外部组织就更容易“改变过去”，用全新的方式取代已有的链条。**这一系列的散列可以作为加密的证据，有助于确保一旦将块添加到区块链，它就不能被替换或删除。**

In [3]:
def next_block(last_block):
    this_index = last_block.index + 1
    this_timestamp = date.datetime.now()
    this_data = "Hey! I'm block " + str(this_index)
    this_hash = last_block.hash
    return Block(this_index, this_timestamp, this_data, this_hash)

现在可以创建区块链了。在这次的示例中，区块链本身是一个简单的Python列表。列表的第一个元素是起源块。当然，还需要添加后续的块，因为SnakeCoin是最小的区块链，这里只添加5个新的块。

In [4]:
# Create the blockchain and add the genesis block
blockchain = [create_genesis_block()]
previous_block = blockchain[0]

# How many blocks should we add to the chain
# after the genesis block
num_of_blocks_to_add = 5

# Add blocks to the chain
for i in range(0, num_of_blocks_to_add):
    block_to_add = next_block(previous_block)
    blockchain.append(block_to_add)
    previous_block = block_to_add
    # Tell everyone about it!
    print("Block #{} has been added to the blockchain!".format(block_to_add.index))
    print("Hash: {}\n".format(block_to_add.hash))

Block #1 has been added to the blockchain!
Hash: bdc60c2d209b949c2e560b4a006cceaabcbfe89bbaa305dd129fb6d0faa84c43

Block #2 has been added to the blockchain!
Hash: 21e3de257a8fd9d64d62d40b8869681f0dd05d234915fdc62378687016621185

Block #3 has been added to the blockchain!
Hash: cbcefb396d15e19704d9058206be9b89aab6e2bc14d0e1d68b91376732cbf96e

Block #4 has been added to the blockchain!
Hash: 6aafa7173a0594d780bd621d0bfef1b20e0b6b692cedd38305a22d0418bc3f17

Block #5 has been added to the blockchain!
Hash: 2474a8182811ea3559507eeae34de26de69887d565a9fbd9427d81c5283375b6



为了使SnakeCoin规模达到今天生产区块链的规模，必须添加更多的功能，比如服务器层，以跟踪多台机器上的链变化，以及在给定的时间段内限制添加的块数量的工作算法。